#Introduction
Using this Python notebook you will:

Understand three Chicago datasets
Load the three datasets into three tables in a SQLIte database
Execute SQL queries to answer assignment questions

#Understand the datasets
##1. Socioeconomic Indicators in Chicago
This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:

https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2

##2. Chicago Public Schools
This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:

https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t

##3. Chicago Crime Data
This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days.

A detailed description of this dataset and the original dataset can be obtained from the Chicago Data Portal at:

https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2

In [ ]:
!pip install ipython-sql prettytable

import prettytable

prettytable.DEFAULT = 'DEFAULT'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.3 MB/s eta 0:00:00


In [ ]:
# ✅ Import the required libraries
import pandas as pd
import sqlite3

# ✅ Establish the SQLite database connection
conn = sqlite3.connect("FinalDB.db")

# ✅ Optional: Create a cursor object for executing raw SQL queries
cursor = conn.cursor()

print("Database connection established!")

Database connection established!


In [ ]:
#Load the SQL magic module
%load_ext sql

In [ ]:
# Chicago Census Data
census_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv"
census_df = pd.read_csv(census_url)

# Chicago Public Schools
schools_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv"
schools_df = pd.read_csv(schools_url)

# Chicago Crime Data
crime_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv"
crime_df = pd.read_csv(crime_url)

In [ ]:
#Establish a connection between SQL magic module and the database FinalDB.db
%sql sqlite:///FinalDB.db

In [ ]:
print(census_df.head())
print(schools_df.head())
print(crime_df.head())

   COMMUNITY_AREA_NUMBER COMMUNITY_AREA_NAME  PERCENT_OF_HOUSING_CROWDED  \
0                    1.0         Rogers Park                         7.7   
1                    2.0          West Ridge                         7.8   
2                    3.0              Uptown                         3.8   
3                    4.0      Lincoln Square                         3.4   
4                    5.0        North Center                         0.3   

   PERCENT_HOUSEHOLDS_BELOW_POVERTY  PERCENT_AGED_16__UNEMPLOYED  \
0                              23.6                          8.7   
1                              17.2                          8.8   
2                              24.0                          8.9   
3                              10.9                          8.2   
4                               7.5                          5.2   

   PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA  \
0                                          18.2   
1                               

In [ ]:
census_df.to_sql("CENSUS_DATA", conn, if_exists='replace', index=False)
schools_df.to_sql("CHICAGO_PUBLIC_SCHOOLS", conn, if_exists='replace', index=False)
crime_df.to_sql("CHICAGO_CRIME_DATA", conn, if_exists='replace', index=False)

# Commit the changes to the database
conn.commit()

In [ ]:
%sql SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///FinalDB.db
Done.


name
CENSUS_DATA
CHICAGO_PUBLIC_SCHOOLS
CHICAGO_CRIME_DATA


###Find the total number of crimes recorded in the CRIME table.

In [ ]:
%sql SELECT COUNT(*) FROM CHICAGO_CRIME_DATA;

 * sqlite:///FinalDB.db
Done.


COUNT(*)
533


###List community area names and numbers with per capita income less than 11000.

In [ ]:
# SQL query to find community areas with per capita income less than 11000
query = """
SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER
FROM CENSUS_DATA
WHERE PER_CAPITA_INCOME < 11000;
"""

# Execute the query and fetch results
result = cursor.execute(query).fetchall()

# Display the result
for row in result:
    print(row)

('West Garfield Park', 26.0)
('South Lawndale', 30.0)
('Fuller Park', 37.0)
('Riverdale', 54.0)


###List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [ ]:
# SQL query to find case numbers and descriptions for crimes involving minors
query = """
SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION
FROM CHICAGO_CRIME_DATA
WHERE DESCRIPTION LIKE '%minor%';
"""

# Execute the query and fetch results
result = cursor.execute(query).fetchall()

# Display the result
for row in result:
    print(row)

('HL266884', 'LIQUOR LAW VIOLATION', 'SELL/GIVE/DEL LIQUOR TO MINOR')
('HK238408', 'LIQUOR LAW VIOLATION', 'ILLEGAL CONSUMPTION BY MINOR')


###List all kidnapping crimes involving a child?

In [ ]:
# SQL query to find case numbers and descriptions for kidnapping crimes involving children
query = """
SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION
FROM CHICAGO_CRIME_DATA
WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%';
"""

# Execute the query and fetch results
result = cursor.execute(query).fetchall()

# Display the result
for row in result:
    print(row)

('HN144152', 'KIDNAPPING', 'CHILD ABDUCTION/STRANGER')


###List the kind of crimes that were recorded at schools. (No repetitions)

In [ ]:
# SQL query to find distinct crime types recorded at schools
query = """
SELECT DISTINCT PRIMARY_TYPE
FROM CHICAGO_CRIME_DATA
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%';
"""

# Execute the query and fetch results
result = cursor.execute(query).fetchall()

# Display the result
for row in result:
    print(row)

('BATTERY',)
('CRIMINAL DAMAGE',)
('NARCOTICS',)
('ASSAULT',)
('CRIMINAL TRESPASS',)
('PUBLIC PEACE VIOLATION',)


###List the type of schools along with the average safety score for each type.

In [ ]:
# SQL query to find school types (Elementary, Middle, or High) along with average safety score
query = """
SELECT "Elementary, Middle, or High School", AVG(SAFETY_SCORE) AS AVG_SAFETY_SCORE
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle, or High School";
"""

# Execute the query and fetch results
result = cursor.execute(query).fetchall()

# Display the result
for row in result:
    print(row)

('ES', 49.52038369304557)
('HS', 49.62352941176471)
('MS', 48.0)


###List 5 community areas with highest % of households below poverty line

In [ ]:
# SQL query to find top 5 community areas with the highest % of households below the poverty line
query = """
SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5;
"""

# Execute the query and fetch results
result = cursor.execute(query).fetchall()

# Display the result
for row in result:
    print(row)

('Riverdale', 56.5)
('Fuller Park', 51.2)
('Englewood', 46.6)
('North Lawndale', 43.1)
('East Garfield Park', 42.4)


###Which community area is most crime prone? Display the coumminty area number only.

In [ ]:
# SQL query to find the community area with the most crimes
query = """
SELECT COMMUNITY_AREA_NUMBER, COUNT(*) AS CRIME_COUNT
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY CRIME_COUNT DESC
LIMIT 1;
"""

# Execute the query and fetch results
result = cursor.execute(query).fetchall()

# Display the result
for row in result:
    print(row[0])  # Displaying only the community area number

25.0


###Use a sub-query to find the name of the community area with highest hardship index

In [ ]:
# SQL query to find the community area with the highest hardship index
query = """
SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE HARDSHIP_INDEX = (
    SELECT MAX(HARDSHIP_INDEX)
    FROM CENSUS_DATA
);
"""

# Execute the query and fetch results
result = cursor.execute(query).fetchall()

# Display the result
for row in result:
    print(row[0])  # Displaying the name of the community area

Riverdale


###Use a sub-query to determine the Community Area Name with most number of crimes?

In [ ]:
# SQL query to find the community area with the most number of crimes
query = """
SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM CHICAGO_CRIME_DATA
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC
    LIMIT 1
);
"""

# Execute the query and fetch results
result = cursor.execute(query).fetchall()

# Display the result
for row in result:
    print(row[0])  # Displaying the name of the community area

Austin


#**AUTHOR**
**IMAD** **HUSAIN**